In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Protenus Sales Training Data.csv', low_memory=False)
df['Snapshot_Date'] = pd.to_datetime(df['Snapshot_Date'], format='%m/%d/%y')
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format='%m/%d/%y')
df['Opportunity_Close_Date'] = pd.to_datetime(df['Opportunity_Close_Date'], format='%m/%d/%y')

In [3]:
def get_quarter_year_from_date(date):
    return 'Q' + str(date.quarter) + '-' + str(date.year)

df['Snapshot_Quarter'] = df['Snapshot_Date'].apply(get_quarter_year_from_date)
df['Opportunity_Close_Quarter'] = df['Opportunity_Close_Date'].apply(get_quarter_year_from_date)

In [4]:
df['true_close_q'] = None

for index, curr_row in df.iterrows():
    curr_id = curr_row['Opportunity_ID']
    pred_close_q = curr_row['Opportunity_Close_Quarter']

    curr_opp_df = df[df['Opportunity_ID'] == curr_id].copy()

    closed_row = curr_opp_df[(curr_opp_df['Opportunity_Stage'] == 'Closed Won') | \
                            (curr_opp_df['Opportunity_Stage'] == 'Closed Lost')]
    
    if len(closed_row) > 0:
        true_close_q = closed_row['Opportunity_Close_Quarter'].values[0]
        df.loc[index, 'true_close_q'] = true_close_q

df['pred_q_correct'] = df['true_close_q'] == df['Opportunity_Close_Quarter']
df['pred_q_correct'] = df['pred_q_correct'].astype(int)

In [5]:
def add_one_quarter(date_str):
    quarter = int(date_str[1])
    year = int(date_str[3:])
    if quarter == 4:
        quarter = 1
        year += 1
    else:
        quarter += 1
    return 'Q' + str(quarter) + '-' + str(year)

df['next_quarter'] = df['Snapshot_Quarter'].apply(add_one_quarter)
df['closed_next_q'] = df['Opportunity_Close_Quarter'] == df['next_quarter']
df['closed_next_q'] = df['closed_next_q'].astype(int)
df['pred_q_plus_1_correct'] = (df['true_close_q'] == df['next_quarter']) | (df['pred_q_correct'] == 1)

In [6]:
# Apply one-hot encoding to categorical columns
dummy_cols = ['Opportunity_Stage', 'Opportunity_Type', 'Opportunity_Product', 'Account_Electronic_Medical_Record_System', 'Account_Hospital_Type']
df_dummies = pd.get_dummies(df, columns=dummy_cols)

In [9]:
# Train a model to predict whether closed in next quarter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# X = df_dummies.drop(['Opportunity_ID', 'Opportunity_Stage', 'Opportunity_Created_Date', 'Opportunity_Close_Date', 'Snapshot_Date', 'Snapshot_Quarter', 'Opportunity_Close_Quarter', 'true_close_q', 'next_quarter', 'closed_next_q'], axis=1)
# Get cols in df_dummies that loook like dummy cols
dummy_cols = [col for col in df_dummies.columns if col.startswith('Opportunity_Stage') or col.startswith('Opportunity_Type') or col.startswith('Opportunity_Product') or col.startswith('Account_Electronic_Medical_Record_System') or col.startswith('Account_Hospital_Type')]
X = df_dummies[dummy_cols]
y = df_dummies['next_quarter']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

# Evaluate whether model is better than random guessing
# Random guessing accuracy
y_test.value_counts()[0] / len(y_test)


0.09214245336348219


/var/folders/ts/q1y61_j510l6blv4tx_7hplc0000gn/T/ipykernel_89190/1577116544.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_test.value_counts()[0] / len(y_test)


0.09044657998869418